In [1]:
import pandas as pd
import numpy as np
import csv
from sklearn.neighbors import KNeighborsRegressor
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,mean_absolute_error

In [ ]:
with open('/content/dailymonthly.csv', encoding='utf-8-sig') as file_obj:
    # Create reader object by passing the file object to DictReader method
    reader_obj = csv.DictReader(file_obj)

    # Open a file to save the results
    with open("/content/KNN_IMD_RDI.csv", "w+") as f:
        # Define a csv writer object
        writer = csv.writer(f)
        writer.writerow(['Loc', 'mse', 'rmse', 'mae'])

        # Iterate over each row in the csv file using reader object
        for row in reader_obj:
            data = row['daily']
            print(row)

            imd = pd.read_csv('/content/IMD/' + data + '.csv').drop(['Unnamed: 0', 'Year'], axis=1)
            imd = imd.replace([np.inf, -np.inf], np.nan)

            X = imd.iloc[5:, 0:4]
            Y = imd.iloc[5:, 3:4]

            imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
            imputer.fit(Y)
            Y = imputer.transform(Y)
            X['RDIn'] = Y
            Y = X.iloc[:, 3]
            Ya = Y.iloc[10:]

            X_train = np.zeros((30, 10))
            X_test = np.zeros((20, 10))
            for i in range(30):
                X_train[i, :] = Y[i:i + 10]
            for i in range(20):
                X_test[i, :] = Y[i + 30:i + 30 + 10]

            X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
            Y_train = np.array(Ya.iloc[0:30])
            Y_train = Y_train.reshape(Y_train.shape[0], 1, 1)

            X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
            Y_test = np.array(Ya.iloc[30:])
            Y_test = Y_test.reshape(Y_test.shape[0], 1, 1)

            X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

            k=5
            model = KNeighborsRegressor(
                n_neighbors=k
                )
            model.fit(
                X_train,
                Y_train
                )
            Y_pred = model.predict(X_test)

            mse = mean_squared_error(Y_test, Y_pred)
            rmse = np.sqrt(mse)
            mae = mean_absolute_error(Y_test, Y_pred)

            print("KNN model evaluation report:")
            print("----------------------------")
            print("Mean Squared Error:", mse)
            rmse = round((np.sqrt(mse)), 5)
            print("Root Mean Squared Error:", rmse)
            print("Mean Absolute Error:", mae)
            print("\n \n")

            # Write the results to the file
            writer.writerow([data, mse, rmse, mae])